In [45]:
import ast

def translate_gp_format(genetic_program): 
    """

    Tree parsing algorithm 
    Parameters:
    genetic_program: Genetic program
    
    Returns:
    Translated Genetic program in simplest polynomial format
    """
    expression = str(genetic_program)

    # replace the numerical constants with a symbol 
    

    def transform_expression(expression):
    # Parse the expression into an AST
        tree = ast.parse(expression, mode='eval')
        def transform(node):
            if isinstance(node, ast.Call):
                func_name = node.func.id
                args = [transform(arg) for arg in node.args]
                return [func_name] + args
            elif isinstance(node, ast.Name):
                return node.id
            elif isinstance(node, ast.Constant):
                return node.value
            else:
                raise ValueError(f"Unsupported AST node type: {type(node)}")

        return transform(tree.body)

    transformed_expression = transform_expression(expression)

    def simplify_expression(transformed_expression):
        # Base case: if the expression is not a list, return it as is
        if not isinstance(transformed_expression, list):
            return transformed_expression
        
        # Recursive case: process the nested list
        operator = transformed_expression[0]
        operands = transformed_expression[1:]
        
        # Simplify each operand
        simplified_operands = [simplify_expression(operand) for operand in operands]
        
        # Combine the simplified operands based on the operator
        if operator == 'add':
            return f"({simplified_operands[0]} + {simplified_operands[1]})"
        elif operator == 'mul':
            return f"({simplified_operands[0]} * {simplified_operands[1]})"
        elif operator == 'sub':
            return f"({simplified_operands[0]} - {simplified_operands[1]})"
        elif operator == 'div':
            return f"({simplified_operands[0]} / {simplified_operands[1]})"
        else:
            raise ValueError(f"Unknown operator: {operator}")

    return simplify_expression(transformed_expression)

In [89]:
# Examle usage 
genetic_program = 'sub(add(0.9, mul(X1, 5)), mul(sub(X1, X0), add(X0, X1)))'
translated_gp = translate_gp_format(genetic_program)
display(translated_gp)

'((0.9 + (X1 * 5)) - ((X1 - X0) * (X0 + X1)))'